In [47]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM
from transformers import AutoTokenizer

In [48]:
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [49]:
dir_ = "./model"

In [50]:
model = AutoModelForCausalLM.from_pretrained(dir_,local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [51]:
telegram_token ='1905825462:AAFM0JI-F2RlvENDAOQV-7zumbTcwW6ODv4'

In [52]:
welcome_text = 'Добрый день!! Вас приветствует бот, который генерирует сказки.  Для его обучения были взяты следующие сказки А.С. Пушкина:\
Руслан и Людмила, Сказка о золотом петушке, Сказка о мёртвой царевне, Сказка о попе и работнике его Балде\
, Сказка о царе Салтане. Задайте начальную фразу, например "И днем и ночью кот ученый", и получите продолжение длинной около 100 слов. Ответ занмает около 20-30 секунд. Спасибо!'

In [53]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text(welcome_text)

In [54]:
def Fairy_tale(update: Update, context: CallbackContext):
    prefix = update.message.text
    
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    size = tokens['input_ids'].shape[1]
    output = model.generate(
    **tokens, 
    #end_token=end_token_id,
    do_sample=False, 
    max_length=size+100, 
    repetition_penalty=5., 
    temperature=0.4,
    num_beams=10,)

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    result =  prefix + result
    
    
    update.message.reply_text(result)

In [55]:
updater = Updater(telegram_token, use_context=True)

In [56]:
updater.dispatcher.add_handler(CommandHandler('start', startCommand))

In [57]:
updater.dispatcher.add_handler(MessageHandler(Filters.text, Fairy_tale))

In [ ]:
updater.start_polling()
updater.idle()

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
